<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Understanding-MAGI-output" data-toc-modified-id="Understanding-MAGI-output-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Understanding MAGI output</a></span></li><li><span><a href="#My-approach-to-magi_results.csv" data-toc-modified-id="My-approach-to-magi_results.csv-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>My approach to magi_results.csv</a></span></li><li><span><a href="#Get-MetaCyc-compound-ids" data-toc-modified-id="Get-MetaCyc-compound-ids-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get MetaCyc compound ids</a></span></li></ul></div>

# Import libraries

In [1]:
import os
import pandas as pd
import numpy as np

# Understanding MAGI output

Explanation of [MAGI output](https://magi.nersc.gov/help/#outputs), and what to do with [magi_results.csv](https://magi.nersc.gov/tutorial/#step4-4)

- `compound_score` == if user doesn't provide a score, this will be 1.0
- `level` describes how far into the chemical similarity network MAGI went to connect the compound to the gene
    - a value of 1 == used chemical network similarity
    - a value of 0 == chemical network was not used
    - no value == no compound associated with the gene
- `homology_score` reciprocal homology score between the reaction in `database_id_r2g` and the gene.
    - a value of 400 is the maximum possible homology score, which means that both compound and genes were connected to the reaction with perfect homology
- `reciprocal_score` is a direct representation of whether or not the reaction-to-gene and gene-to-reaction homology searches converged on the same reactions or not, and it is determined by using the top BLAST result
    - a value of 2 == reciprocal agreement
    - a value of 1 == reciprocal closeness because the E-score was close enough
    - a value of 0.1 == only one of the two searches resulted in a reaction
    - a value of 0.01 == no reciprocal agreement
- `database_id_g2r` == reactions that a gene product can catalyze, which mean a gene-centric view
- `database_id_r2g` == reactions that associated with both gene and compound, which means a compound-centric view


There are three approaches to explore results:

1. Stich biochemical pathways. Filter the table to only show rows pertaining to a compound, look at the `database_id_r2g` for all the reactions, and `gene_id` for a list of genes associated with that compound

2. Curating a metabolic model:
    1. To see all the compounds a gene was connected to: filter the table to only show one gene, and look at the `database_id_g2r`
    2. To see all the evidence for a particular biochemical reaction: filter the `database_id_g2r` or `database_id_r2g` to only show one reaction, and see all the compounds of that reaction were observed as well as genes associated with that reaction
    
    
------------

# My approach to magi_results.csv

My approach will be to:

1. filter based on `MAGI_score` > 1, `homology_score` == 400, `reciprocal_score` > 1, and `note` column == direct
2. filter based on gene-compound direct associations (when the `neighbor` column is empty), and indirect association (when `neighbor` column is not empty and note == direct)

In [2]:
df = pd.read_csv('magi_output/magi_results.csv')

df = df.sort_values('MAGI_score', ascending=False).reset_index(drop=True)

print(df.shape)

df['neighbor'] = df['neighbor'].replace(np.nan, '')

df.head()

(183089, 14)


,MAGI_score,gene_id,original_compound,neighbor,note,compound_score,level,homology_score,reciprocal_score,reaction_connection,e_score_r2g,database_id_r2g,e_score_g2r,database_id_g2r
0,6.332446,Qrob_P0760750.2,GDVRUDXLQBVIKP-HQHREHCSSA-N,,direct,1.0,0.0,400.0,2.0,2.01,200.0,GALLATE-1-BETA-GLUCOSYLTRANSFERASE-RXN,200.0,GALLATE-1-BETA-GLUCOSYLTRANSFERASE-RXN
1,6.332446,Qrob_P0276600.2,HDYANYHVCAPMJV-NTWGTYGRSA-N,,flat tautomer,1.0,0.0,400.0,2.0,2.01,200.0,RHEA:16327,200.0,RHEA:16327
2,6.332446,Qrob_P0276600.2,HDYANYHVCAPMJV-PCMFUMOBSA-N,,flat tautomer,1.0,0.0,400.0,2.0,2.01,200.0,RHEA:16327,200.0,RHEA:16327
3,6.332446,Qrob_P0276600.2,HDYANYHVCAPMJV-PCMFUMOBSA-N,,flat tautomer,1.0,0.0,400.0,2.0,2.01,200.0,RXN-3,200.0,RXN-3
4,6.332446,Qrob_P0041490.2,HDYANYHVCAPMJV-PCMFUMOBSA-N,,flat tautomer,1.0,0.0,400.0,2.0,2.01,200.0,RHEA:11404,200.0,RHEA:11404


In [44]:
# filtering
subset_df = df[(df['MAGI_score'] >= 2) &  # (df['homology_score'] == 400) &
               (df['reciprocal_score'] >= 1) &
               (df['note'] == 'direct')].reset_index(drop=True)

# getting the feature label from magi_output/magi_compound_result.csv
ft = pd.read_csv('magi_output/magi_compound_results.csv')
ft = ft[['original_compound', 'label']]

# merge
subset_df = subset_df.merge(ft, on='original_compound')

# direct association
subset_direct = subset_df[subset_df['neighbor'] == ""].reset_index(drop=True)
print(subset_direct.shape)
subset_direct.to_csv('magi_results_direct.csv', index=False)

# indirect association
subset_indirect = subset_df[subset_df['neighbor']!=""].reset_index(drop=True)
print(subset_indirect.shape)
# subset_indirect.to_csv('magi_results_indirect.csv', index = False)

(912, 15)
(0, 15)


# Get MetaCyc compound ids

In [45]:
metacyc = pd.read_csv("metacyc/All_compounds_of_MetaCyc.txt", sep = '\t')

metacyc['InChI-Key'] = metacyc['InChI-Key'].str[:-2]

metacyc

,Compound,InChI-Key
0,CPD-13006,InChIKey=HDEIGSIJUBUHSO-UHFFFAOYSA
1,CPD-12542,InChIKey=SDADNVAZGVDAIM-QTNLNCNHSA
2,CPD-18037,InChIKey=QSLZNGPMBOCIAZ-YEARRENQSA
3,D-SORBITOL-6-P,InChIKey=GACTWZZMVMUKNG-SLPGGIOYSA
4,GALACTITOL-1-PHOSPHATE,InChIKey=GACTWZZMVMUKNG-DPYQTVNSSA
...,...,...
16856,N-ACETYLD-D-MANNOSAMINE-1P,InChIKey=FZLJPEPAYPUMMR-ZTVVOAFPSA
16857,CPD-19506,InChIKey=FMJUTBOFNVTKSJ-UHFFFAOYSA
16858,CPD-18041,InChIKey=FKBKETJRCKZDAM-TVWJAPESSA
16859,CPD-22555,InChIKey=ICOBPPWZFJRDGN-UHFFFAOYSA


In [48]:
direct_compounds = subset_direct[['original_compound',
                                  'label']].drop_duplicates(
                                      ['original_compound',
                                       'label']).reset_index(drop=True)

direct_compounds['original_compound'] = "InChIKey=" + direct_compounds[
    'original_compound'].str[:-2]

direct_compounds = direct_compounds.merge(metacyc,
                                          left_on='original_compound',
                                          right_on='InChI-Key')

direct_compounds.drop('original_compound', axis=1, inplace=True)

direct_compounds.to_csv('magi_compounds_direct.csv', sep='\t', index=False)

direct_compounds

,label,Compound,InChI-Key
0,FT0550,1-O-GALLOYL-BETA-D-GLUCOSE,InChIKey=GDVRUDXLQBVIKP-HQHREHCSSA
1,FT1134,CPD1F-453,InChIKey=JPUKWEQWGBDDQB-QSOFNFLRSA
2,FT0419,CPD-520,InChIKey=REFJWTPEDVJJIY-UHFFFAOYSA
3,FT2130,UDP-GLUCURONATE,InChIKey=HDYANYHVCAPMJV-LXQIFKJMSA
4,FT2130,UDP-D-GALACTURONATE,InChIKey=HDYANYHVCAPMJV-GXNRKQDOSA
5,FT2417,OXIDIZED-GLUTATHIONE,InChIKey=YPZRWBKMTBYPTK-BJDJZHNGSA
6,FT0651,5Z13E-15S-9-ALPHA11-ALPHA15-TRIHY,InChIKey=PXGPLTODNUVGFL-YNNPMVKQSA


Imported this into MetaCyc as a SmartTable.

In MetaCyc's SmartTable, I added structure, reactions that consume and produce such compounds to be used in the discussion. I then, exported the SmartTable as frame_ids (which retains the compound and reaction identifiers) and common names (with common names of everything).